In [14]:
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)
import pandas as pd
import logging
import numpy as np



transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# %%
import dvc.api

df_test = pd.read_json(dvc.api.get_url(
    'datasets/data/query_label/processed/Offshore_Labelled_Query_Classification_Test_V2.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)

# %%
df_tax = pd.read_json(dvc.api.get_url(
    'datasets/data/taxonomy/wish_newtax.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)
df_tax = df_tax[(df_tax.category_path.apply(len) > 0) & (df_tax.category_path.apply(lambda x: len(x.split(' > ')) == 1))]

# %%
LABEL_SET = sorted(df_tax.category_path.str.lower().str.strip().tolist()) + ['unknown']

# %%
LABEL_NAME_TO_ID = {i: ind for ind, i in enumerate(LABEL_SET)}
print(LABEL_NAME_TO_ID)
# %%

# %%
def categories2labels(cats):
    if len(cats) == 0:
        cats = ['unknown']
    labs = [0] * len(LABEL_NAME_TO_ID)
    for c in cats:
        labs[LABEL_NAME_TO_ID[c]] = 1
    return labs


{'apparel accessories': 0, 'automobiles & motorcycles': 1, 'beauty & health': 2, 'cellphones & telecommunications': 3, 'computer & office': 4, 'consumer electronics': 5, 'education & office supplies': 6, 'electronic components & supplies': 7, 'entertainment': 8, 'food': 9, 'furniture': 10, 'hair extensions & wigs': 11, 'home & garden': 12, 'home improvement': 13, 'jewelry & accessories': 14, 'luggage & bags': 15, "men's clothing": 16, 'mother & kids': 17, 'novelty & special use': 18, 'security & protection': 19, 'shoes': 20, 'sports': 21, 'tools': 22, 'toys & hobbies': 23, 'virtual goods': 24, 'watches': 25, "women's clothing": 26, 'unknown': 27}


In [15]:
model = MultiLabelClassificationModel(
    "xlmroberta",
    "outputs/best_model",
)

In [16]:
df_test['labels'] = df_test['query_classification_lists'].apply(
    lambda x: categories2labels([i.lower().strip().split(' > ')[0] for i in x]))
df_test['text'] = df_test['query']

In [17]:
result, model_outputs, wrong_predictions = model.eval_model(
    df_test
)

Running Evaluation:   0%|          | 0/1681 [00:00<?, ?it/s]

In [18]:
result

{'LRAP': 0.8539614235731265, 'eval_loss': 0.07486113636911797}

In [5]:
result

{'LRAP': 0.8318010818326117, 'eval_loss': 0.08628105077280894}

In [19]:
from sklearn.metrics import classification_report

In [ ]:
# print(classification_report(np.array(df_test['labels'].tolist()), model_outputs>.5))

In [21]:
import pandas as pd

In [22]:
df_qcv2 = pd.read_json(
    "../../models/multitask_multimodal_multilingual/version_9/clm-epoch=1-step=2600--wish_labelled_query_offshore_test_V2--test.json", 
    lines=True)

In [23]:
df_qcv2 = df_qcv2.sort_values(['batch_indices', 'rank_indices']).groupby('batch_indices').agg({
    'prediction_decoded': lambda x: [i for i in x], 
    'prob': lambda x: [i for i in x]
})

In [24]:
len(df_test)

13448

In [37]:
recs = []
for i in df_qcv2.to_dict('records'):
    prediction_decoded_confident = []
    for pred, prob in zip(i['prediction_decoded'], i['prob']):
        if prob >= .05:
            prediction_decoded_confident.append(pred)
    i['prediction_decoded_confident'] = prediction_decoded_confident
    recs.append(i)
df_qcv2 = pd.DataFrame(recs)

In [42]:
df_qcv2['labels'] = df_qcv2['prediction_decoded_confident'].apply(
    lambda x: categories2labels([i.lower().strip().split(' > ')[0] for i in x]))

: 

In [ ]:
# print(classification_report(np.array(df_test['labels'].tolist()), np.array(df_qcv2['labels'].tolist())))